In [8]:
import pandas as pd
import sqlite3
from sqlalchemy.engine import create_engine
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
import utils
from sklearn.model_selection import TimeSeriesSplit

In [23]:
target = 'deal_probability'
LIMIT = 100000

In [24]:
avito = sqlite3.connect("avito.db")


In [35]:
train = pd.read_sql('SELECT price,item_seq_number,city,category_name,user_type,deal_probability FROM train WHERE price < 10000', avito)
train = train.dropna()

In [36]:
city = preprocessing.LabelEncoder()
city.fit(train['city'])  
list(city.classes_)
train['city'] = city.transform(train['city'])

categ = preprocessing.LabelEncoder()
categ.fit(train['category_name'])  
list(categ.classes_)
train['category_name'] = categ.transform(train['category_name'])

user = preprocessing.LabelEncoder()
user.fit(train['user_type'])  
list(user.classes_)
train['user_type'] = user.transform(train['user_type'])


In [37]:
#train = pd.get_dummies(train[['category_name', 'user_type']])
train = train[:LIMIT]

In [31]:
#tpot_x = train[['price','item_seq_number','city','category_name','user_type']].values[:30000]
#tpot_y = train['deal_probability'].values[:30000]

In [38]:
tss = TimeSeriesSplit(n_splits=4)
X = (train.drop(target, axis=1)).values
y = train[target].values
train_index, test_index = list(tss.split(X))[-1]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [40]:
train_index

array([    0,     1,     2, ..., 79997, 79998, 79999])

In [41]:
#X_train, X_test, y_train, y_test = train_test_split(tpot_x, tpot_y,
#                                                    train_size=0.75, test_size=0.25)

IndentationError: unexpected indent (<ipython-input-41-03bfd358a04c>, line 2)

In [42]:
#pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
#                                    random_state=42, verbosity=2)

In [43]:
tpot_optimizer = utils.TpotAutoml(mode='regression', max_time_mins=30,
                                  scoring='r2',random_state=23, n_jobs=-1,verbosity=2,
                                  cv=TimeSeriesSplit(n_splits=3))

In [44]:
tpot_optimizer.fit(X_train, y_train)


Optimization Progress: 201pipeline [15:21,  6.89s/pipeline]                   

Generation 1 - Current best internal CV score: 0.14122374055637318


Optimization Progress: 301pipeline [20:41,  5.84s/pipeline]                   

Generation 2 - Current best internal CV score: 0.1418440693839936


Optimization Progress: 401pipeline [26:47,  4.32s/pipeline]                   

Generation 3 - Current best internal CV score: 0.1419814895656163



31.030470033333334 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesRegressor(ZeroCount(StandardScaler(input_matrix)), bootstrap=False, max_features=1.0, min_samples_leaf=17, min_samples_split=3, n_estimators=100)


In [36]:
top_scores = tpot_optimizer.get_top_models(return_scores=True)
print('\ntop cv scores:')
print(top_scores)
print('\ntop models')
print(tpot_optimizer.top_models)
print('\nthe best test score:')
test_score = tpot_optimizer.score(X_test, y_test)
print(test_score)


top cv scores:
OrderedDict([('ExtraTreesRegressor', 0.12938287198748735), ('XGBRegressor', 0.12764902380254345), ('LassoLarsCV', 0.12683514109289584), ('RidgeCV', 0.12411627868558231), ('RandomForestRegressor', 0.12288545880528338), ('DecisionTreeRegressor', 0.1122301523794286), ('KNeighborsRegressor', 0.05007212134948704), ('GradientBoostingRegressor', 0.04064309850213299), ('LinearSVR', 0.013057045884839491), ('ElasticNetCV', 0.01302112305782488)])

top models
OrderedDict([('ExtraTreesRegressor', {'generation': 'INVALID', 'mutation_count': 2, 'crossover_count': 0, 'predecessor': ('ExtraTreesRegressor(XGBRegressor(input_matrix, XGBRegressor__learning_rate=0.01, XGBRegressor__max_depth=2, XGBRegressor__min_child_weight=13, XGBRegressor__n_estimators=100, XGBRegressor__nthread=1, XGBRegressor__subsample=0.35000000000000003), ExtraTreesRegressor__bootstrap=True, ExtraTreesRegressor__max_features=0.7000000000000001, ExtraTreesRegressor__min_samples_leaf=9, ExtraTreesRegressor__min_sample